In [ ]:
#AZURE
from azure.identity import DefaultAzureCredential
from azure.identity import ManagedIdentityCredential
from azure.keyvault.secrets import SecretClient
from azure.storage.blob import BlockBlobService
from azure.storage.blob import ContentSettings
import time
#GENERAL
import http.client
import mimetypes
import json
import requests
from pandas import json_normalize
import sys, os
import pandas as pd
import datetime
from copy import deepcopy
#ARCGIS
from arcgis.features.manage_data import dissolve_boundaries
from arcgis.features.find_locations import find_centroids
from arcgis.geometry import from_geo_coordinate_string
from arcgis.geocoding import geocode
from arcgis.geometry import lengths, areas_and_lengths, project
from arcgis.geometry import Point, Polyline, Polygon, Geometry
from arcgis.gis import GIS
import arcgis
from arcgis import geometry 
from arcgis import features
from arcgis.geoanalytics import manage_data
from arcgis.features.manage_data import overlay_layers
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureLayer
from arcgis.features import FeatureLayerCollection

In [ ]:
gis=GIS("https://domain.arcgis.com", "UserName", "Password")

# Download Polygon Feature Class and create a spatia enabled datframe

In [ ]:

item=gis.content.get('Feature Class ID')
l=item.layers[0]
#df=l.query(out_fields='*', return_centroid=True).sdf# Will give Enabled DataFrames with Centroids
#df=l.query(where="ID='123'",out_fields='*', return_centroid=True).sdf#Will give  Enabled DataFrames with Centroids for 123 IN id Field
df=l.query(where="ID ('123','124')",out_fields='*', return_centroid=True).sdf#Will give  Enabled DataFrames with Centroids for 123 AND 124 IN id Field


###  Dissolves downloaded Polygon Feature Class on the named fields and publishes new feature collection Called "findcentroids"

In [ ]:
dissolve_fields=['ID','Name']
point_centroid = dissolve_boundaries(l, dissolve_fields,estimate=True,output_name='findcentroids', future=True)


###  Searches "findcentroids" allowing 80 seconds latency, and returns its portal ID

In [ ]:
#import datetime
item = gis.content.search("findcentroids",item_type="Feature Layer Collection")
while len(item)==0:
    time.sleep(80)
    item = gis.content.search("findcentroids",item_type="Feature Layer Collection")#print (len(gis.content.search("findcentroids",item_type="Feature Layer Collection")))
    print(item[0].id)


In [ ]:
### Using ID acquired above download the findcentroids and computes centroids creating a new feature collection layer called "HealthLyrPolygonToPoint"

In [ ]:
c= gis.content.get('{}'.format(item[0].id)).layers[0]
poly_to_point = find_centroids(c, output_name="HealthLyrPolygonToPoint", future=True)

###  Searches "HealthLyrPolygonToPoint" allowing 80 seconds latency, and returns its portal ID

In [ ]:
items = gis.content.search("HealthLyrPolygonToPoint",item_type="Feature Layer Collection")
while len(items)==0:
    time.sleep(80)
    items = gis.content.search("HealthLyrPolygonToPoint",item_type="Feature Layer Collection")#print (len(gis.content.search("findcentroids",item_type="Feature Layer Collection")))
    print(items[0].id)
  

In [ ]:
pd.set_option("max_rows", None)# Display maximum rows for any datframe in the notebook

In [ ]:
### Using ID acquired above download the HealthLyrPolygonToPoint and creates a spatially enabled dataframe in memory called table

In [ ]:
  
table= gis.content.get('{}'.format(item[0].id)).layers[0].query().sdf

table


# Compute Coordinates from the table's column called SHAPE ***SHAPE is of dtype geometry

In [ ]:
table=table.assign(Long=table.SHAPE.astype(str).apply(lambda x: Point(x).coordinates()).str[0],Lat=table.SHAPE.astype(str).apply(lambda x: Point(x).coordinates()).str[1])
table


# Delete Layers HealthLyrPolygonToPoint and  findcentroids from Portal

In [ ]:
gis.content.search("HealthLyrPolygonToPoint")[0].delete()

In [ ]:
gis.content.search("findcentroids")[0].delete()
